<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import os
import sys
import glob
import netCDF4 as nc4

!dir

ACTAMERICA-mrg05-b200_merge_20170201_R1.nc
ACTAMERICA-mrg05-b200_merge_20170204_R1.nc
ACTAMERICA-mrg05-b200_merge_20170206_R1.nc
ACTAMERICA-mrg05-b200_merge_20170212_R1.nc
ACTAMERICA-mrg05-b200_merge_20170216_R1.nc
ACTAMERICA-mrg05-b200_merge_20170223_R1.nc
ACTAMERICA-mrg05-b200_merge_20170225_R1.nc
ACTAMERICA-mrg05-b200_merge_20170301_R1.nc
ACTAMERICA-mrg05-b200_merge_20170304_R1.nc
ACTAMERICA-mrg05-b200_merge_20170307_R1.nc
ACTAMERICA-mrg05-b200_merge_20171003_R0.nc
ACTAMERICA-mrg05-b200_merge_20171004_R0.nc
ACTAMERICA-mrg05-b200_merge_20171005_R0.nc
ACTAMERICA-mrg05-b200_merge_20171008_R0.nc
ACTAMERICA-mrg05-b200_merge_20171010_R0.nc
ACTAMERICA-mrg05-b200_merge_20171011_R0.nc
ACTAMERICA-mrg05-b200_merge_20171014_R0.nc
ACTAMERICA-mrg05-b200_merge_20171016_R0.nc
ACTAMERICA-mrg05-b200_merge_20171018_R0.nc
ACTAMERICA-mrg05-b200_merge_20171020_R0.nc
ACTAMERICA-mrg05-b200_merge_20171021_R0.nc
ACTAMERICA-mrg05-b200_merge_20171022_R0.nc
ACTAMERICA-mrg05-b200_merge_20171024_R0.nc
ACTAMERICA-

In [3]:
!ls -1 mrg05/*.nc | wc -l

182


In [4]:
ncfiles = glob.glob("mrg05/*.nc")

In [5]:
!ls -1 flags_v2/*.ict | wc -l

181


In [6]:
files = glob.glob("flags_v2/*.ict")
flights = [os.path.basename(f)[25:38] for f in files]
uflights = list(set(flights))
uflightsd = {}

for flt0 in uflights:
    t = []
    for flt1 in files:
        if flt0 in flt1:
            t.append(flt1)
    t = sorted(t)
    if len(t)>1:
        if (("_L1_" in t[-2]) & ("_L2_" in t[-1])):
            uflightsd[flt0] = t
        else:
            uflightsd[flt0] = t[-1]
    else:
        uflightsd[flt0] = t[0]

uflights

['B200_20180412',
 'B200_20180414',
 'C130_20170204',
 'C130_20180427',
 'B200_20180518',
 'B200_20160812',
 'B200_20160816',
 'B200_20170217',
 'C130_20171102',
 'B200_20171004',
 'B200_20180504',
 'C130_20170302',
 'B200_20180429',
 'B200_20180515',
 'B200_20160828',
 'C130_20160819',
 'C130_20160810',
 'B200_20180419',
 'B200_20160718',
 'B200_20171024',
 'B200_20171010',
 'C130_20170215',
 'C130_20170307',
 'B200_20170308',
 'C130_20160820',
 'C130_20160719',
 'B200_20180520',
 'C130_20171021',
 'B200_20171008',
 'B200_20170227',
 'B200_20160727',
 'B200_20160722',
 'B200_20160820',
 'B200_20160808',
 'B200_20170204',
 'C130_20180512',
 'B200_20171022',
 'B200_20180420',
 'C130_20180518',
 'C130_20171106',
 'C130_20180509',
 'C130_20170226',
 'B200_20171018',
 'C130_20160725',
 'C130_20160804',
 'C130_20160721',
 'C130_20170301',
 'C130_20171008',
 'B200_20180516',
 'B200_20171105',
 'B200_20160814',
 'B200_20170212',
 'B200_20180430',
 'B200_20170215',
 'B200_20171030',
 'C130_201

In [7]:
uflightsd2 = {}
for k, v in uflightsd.items():
    file_ = None
    for f in ncfiles:
        fcomp = os.path.basename(f).split("_")
        aircraft = fcomp[0].split("-")[-1].upper()
        date = fcomp[-2]  
        if ((aircraft in k) & (date in k)):
            file_ = f
    uflightsd2[k] = (file_, v)

In [8]:
import json

with open("matches.json", "w") as f:
    f.write(json.dumps(uflightsd2, indent=4))

In [14]:
for f in flights:
    if f not in uflightsd2:
        print(f)

In [19]:
len(list(set(flights)))

171

# Edit

In [ ]:
import pandas as pd

def read_multileg(icartt):
    dfs = {}
    for ict in icartt:
        with open(ict, "r") as f:
            headerrow = int(f.readlines()[0].split(",")[0])-1
        dfs[ict] = pd.read_csv(ict, header=headerrow, delimiter=",")
    df1, df2 = dfs.values()
    df1 = df1.append(df2)
    df1.columns = [c.strip() for c in list(df1.columns)]
    return df1
    
def read_singleleg(icartt):
    with open(icartt, "r") as f:
        headerrow = int(f.readlines()[0].split(",")[0])-1
    df = pd.read_csv(icartt, header=headerrow, delimiter=",")
    df.columns = [c.strip() for c in list(df.columns)]
    return df

def read_flight(flight_tuple):
    icartt = flight_tuple[1]
    flight_tuple2 = [flight_tuple[0], None]
    if type(icartt) is list:
        flight_tuple2[1] = read_multileg(icartt)
    else:
        flight_tuple2[1] = read_singleleg(icartt)
    return flight_tuple2


files = glob.glob("flags_v2/*.ict")
flights = [os.path.basename(f)[25:38] for f in files]
uflights = list(set(flights))
uflightsd = {}

for flt0 in uflights:
    t = []
    for flt1 in files:
        if flt0 in flt1:
            t.append(flt1)
    t = sorted(t)
    if len(t)>1:
        if (("_L1_" in t[-2]) & ("_L2_" in t[-1])):
            uflightsd[flt0] = t
        else:
            uflightsd[flt0] = t[-1]
    else:
        uflightsd[flt0] = t[0]

print(uflightsd)

uflightsd2 = {}
for k, v in uflightsd.items():
    file_ = None
    for f in ncfiles:
        fcomp = os.path.basename(f).split("_")
        aircraft = fcomp[0].split("-")[-1].upper()
        date = fcomp[-2]  
        if ((aircraft in k) & (date in k)):
            file_ = f
    uflightsd2[k] = (file_, v)

print(uflightsd2)
    
uflightsd3 = {}
for k, v in uflightsd2.items():
    try:
        uflightsd3[k] = read_flight(v)
    except:
        uflightsd3[k] = None
        print(k)
        pass
    
print(uflightsd3)

In [ ]:
UNEQUAL = []
FAILS =[]
for flight, data in uflightsd3.items():
    fname = os.path.basename(data[0])
    with nc4.Dataset(data[0]) as src, nc4.Dataset(fname, "w") as dst:
        
        if data[1].index.size == src.dimensions["time"].size:
        
            # copy global attributes all at once via dictionary
            dst.setncatts(src.__dict__)

            # copy dimensions
            for name, dimension in src.dimensions.items():
                dst.createDimension(
                    name, (len(dimension) if not dimension.isunlimited() else None))

            # copy all file data except for the excluded
            for name, variable in src.variables.items():
                try:
                    x = dst.createVariable(
                        name, 
                        variable.datatype, 
                        variable.dimensions, 
                        fill_value=variable._FillValue,
                        zlib=True)
                    del src[name].__dict__["_FillValue"]
                    dst[name].setncatts(src[name].__dict__)
                except:
                    x = dst.createVariable(name, variable.datatype, variable.dimensions, zlib=True)
                    dst[name].setncatts(src[name].__dict__)
                dst[name][:] = src[name][:]
                # copy variable attributes all at once via dictionary
                

    
            try:

                FlightPatternFlag = dst.createVariable('FlightPatternFlag', 'int32', ('time'), fill_value=-999999, zlib=True)
                FlightPatternFlag.long_name = "Flight pattern description"
                FlightPatternFlag.coordinates = "time LATITUDE LONGITUDE GPS_ALT"
                FlightPatternFlag.flag_meanings = "frontal pre-frontal_fair post-frontal_fair fair other"
                FlightPatternFlag.valid_range = (0, 4)
                FlightPatternFlag.flag_values = (0, 1, 2, 3, 4)
                FlightPatternFlag[:] = data[1]["Flight_flag"].fillna(-999999).astype(int).to_numpy()

                if "ManeuverFlag" not in list(dst.variables.keys()):
                    ManeuverFlag = dst.createVariable('ManeuverFlag', 'int32', ('time'), fill_value=-999999,zlib=True)
                    ManeuverFlag.long_name = "data flag for sampling based on aircraft maneuver"
                    ManeuverFlag.coordinates = "time LATITUDE LONGITUDE GPS_ALT"
                    ManeuverFlag.flag_meanings = "on_ground take_off in-line_ascent in-line_descent spiral_up spiral_down constant_alt_legs landing"
                    ManeuverFlag.valid_range = (0, 7)
                    ManeuverFlag.flag_values = (0, 1, 2, 3, 4, 5, 6, 7)
                    ManeuverFlag[:] = data[1]["Maneuver_flag"].fillna(-999999).astype(int).to_numpy()

                ManeuverFlagQC = dst.createVariable('ManeuverFlagQC', 'int32', ('time'), fill_value=-999999,zlib=True)
                ManeuverFlagQC.long_name = "Low or average confidence in ManeuverFlag"
                ManeuverFlagQC.coordinates = "time LATITUDE LONGITUDE GPS_ALT"
                ManeuverFlagQC.flag_meanings = "low_confidence average_confidence"
                ManeuverFlagQC.valid_range = (0, 1)
                ManeuverFlagQC.flag_values = (0, 1)
                ManeuverFlagQC[:] = data[1]["Maneuver_flagQC"].fillna(-999999).astype(int).to_numpy()

                BoundaryLayerFreeTroposphereFlag = dst.createVariable('BoundaryLayerFreeTroposphereFlag', 'int32', ('time'), fill_value=-999999, zlib=True)
                BoundaryLayerFreeTroposphereFlag.long_name = "Flag indicating whether aircraft is grounded, in boundary layer, or in free troposphere"
                BoundaryLayerFreeTroposphereFlag.coordinates = "time LATITUDE LONGITUDE GPS_ALT"
                BoundaryLayerFreeTroposphereFlag.flag_meanings = "on_ground boundary_layer free_troposphere"
                BoundaryLayerFreeTroposphereFlag.valid_range = (0, 2)
                BoundaryLayerFreeTroposphereFlag.flag_values = (0, 1, 2)
                BoundaryLayerFreeTroposphereFlag[:] = data[1]["BL_FT_flag"].fillna(-999999).astype(int).to_numpy()

                WarmColdAirFlag = dst.createVariable('WarmColdAirFlag', 'int32', ('time'), fill_value=-999999, zlib=True)
                WarmColdAirFlag.long_name = "Flag indicating whether aircraft is in warm or cold air, or fair conditions."
                WarmColdAirFlag.coordinates = "time LATITUDE LONGITUDE GPS_ALT"
                WarmColdAirFlag.flag_meanings = "fair cold warm"
                WarmColdAirFlag.valid_range = (0, 2)
                WarmColdAirFlag.flag_values = (0, 1, 2)
                WarmColdAirFlag[:] = data[1]["Air_flag"].fillna(-999999).astype(int).to_numpy()

            except Exception as e:
                FAILS.append((flight,e))
                print(flight)
                print(e)
        else:
            UNEQUAL.append(flight)




{'title': 'Merged data file for ACT-AMERICA 2018, Flight 19 (20180508), on the b200 platform. Data is merged to PFP file timeline.',
 'mission_name': 'NASA ACT-AMERICA 2018 Summer Campaign',
 'flight_date': '2018-05-08',
 'platform': 'NASA B200 Aircraft',
 'location': 'Latitude, Longitude, and Altitude included in data records',
 'associated_data': 'N/A',
 'instrument_info': 'N/A',
 'data_info': 'Please see PI data file for explanation of ULOD and LLOD.',
 'uncertainty': 'See individual files for uncertainty information.',
 'ulod_flag': '-777777',
 'ulod_value': 'N/A',
 'llod_flag': '-888888',
 'llod_value': 'N/A',
 'project_info': 'ACT-America',
 'comments': 'This merge was created using data in the NASA ACT-AMERICA archive as of 10/12/2018. In most cases, variable names have been kept identical to those submitted in the raw data files. However, in some cases, names have been changed  (e.g., to eliminate duplication)  Units have been standardized throughout the merge. Contact michael.

In [56]:
import os
import glob
import json
import yaml
import numpy as np
import netCDF4 as nc4


def get_variable(variable):
    """
    Returns a dictionary describing the variables in a netCDF file.
    """
    
    structure = {}
    
    # Add the dimensions and attributes of the variables in a loop.

        
    return structure


def get_netcdf_dataset_structure(dataset):
    """
    Makes a JSON (Panoply-like) representation of netCDF structure. 
    Groups are also dataset constructs in netCDF4 python.
    
    Parameters
    ----------
    
    Returns
    -------
    
    """
    
    # netCDF structure as a dictionary. 
    structure = {
        'groups': {},
        'variables': {}, 
        'attributes': dataset.__dict__,
    }
    
    # VARIABLES: Add this dataset's variables to structure dictionary.
    for name, variable in dataset.variables.items():
        
        structure['variables'][name] = {
            'dimensions': variable.dimensions,
            'attributes': variable.__dict__,
        }

    # GROUPS: are also datasets. Loop and repeatedly call this function.
    for name, group in dataset.groups.items():
        structure['groups'][name] = get_netcdf_dataset_structure(group)
    
    return structure


def get_netcdf_file_structure(netcdf_file_path: str):
    '''
    
    Parameters
    ----------
    
    Returns
    -------
    
    '''
    
    # Read netCDF and get dictionary structure.
    with nc4.Dataset(netcdf_file_path) as ds:
        structure = get_netcdf_dataset_structure(ds)
    
    # Add dimensions to the dictionary structure.
    for name, dimension in dataset.dimensions.items():

        structure['dimensions'][name] = {
            'UNLIMITED': dimension.isunlimited(),
            'size': dimension.size()
        }
    
    return structure


def construct_PATH_OUTPUT(input_file_path: str, PATH_OUTPUT: str, output_file_extension: str):
    '''
    Simple path constructor.
    
    Parameters
    ----------
    
    Returns
    -------
    
    '''

    # Get output file basename.
    output_file_basename = os.path.splitext(os.path.basename(input_file_path))[0]
    
    # Get output file name.
    output_file_name = "{}.{}".format(output_file_basename, output_file_extension)

    # Use os module to join path.
    output_file_path = os.path.join(PATH_OUTPUT, output_file_name)

    return output_file_path


class NumpyEncoder(json.JSONEncoder):
    '''
    A well known hack for serializing numpy types to JSON.
    '''
    numpy_ints = (  # numpy integer data types.
        np.int_, np.intc, np.intp, np.int8, np.int16, np.int32, 
        np.int64, np.uint8, np.uint16, np.uint32, np.uint64)
    numpy_flts = (  # numpy float data types.
        np.float_, np.float16, np.float32, np.float64)
    def default(self, obj):
        if isinstance(obj, self.numpy_ints):
            return int(obj)
        elif isinstance(obj, self.numpy_flts):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def _write_numpy_yaml(dictionary: dict, output: str):
    '''
    '''

    # Dump to YAML string.
    dict_yaml = yaml.dump(dictionary)  ##, indent=2)) #, cls=NumpyJsonEncoder))

    # Write output YAML.
    with open(output, "w") as f:
        f.write(dict_yaml)

    
def _write_numpy_json(dictionary: dict, output: str):
    '''
    
    Parameters
    ----------
    
    Returns
    -------

    '''

    # Dump to JSON string.
    dict_json = json.dumps(dictionary, indent=2, cls=NumpyEncoder)

    # And write output JSON.
    with open(output, "w") as f:
        f.write(dict_json)


def get_netcdf_file_structure(
    input_path_netcdf: str, 
    PATH_OUTPUT_json:str=None, 
    PATH_OUTPUT_yaml:str=None):
    '''
    Get the complete hierarchical structure of a netCDF file as a Python dict.
    
    Parameters
    ----------
    
    Returns
    -------
    
    '''
    
    # Open the nteCDF file.
    netcdf_dataset = nc4.Dataset(input_path_netcdf)
   
    # Get the structure of the dataset root.
    root_structure = get_netcdf_dataset_structure(netcdf_dataset)
    
    # Close the open netCDF dataset.
    netcdf_dataset.close()
       
    # If JSON path was given as arg, write to disk.
    if PATH_OUTPUT_json is not None:
        _write_numpy_json(root_structure, PATH_OUTPUT_json)

    # If YAML path was given as arg, write to disk.
    if PATH_OUTPUT_yaml is not None:
        _write_numpy_yaml(root_structure, PATH_OUTPUT_yaml)




netcdf = glob.glob("/data/actamerica/ACTAMERICA_Merge/data/nc/*.nc*")

test_in_nc = '/data/actamerica/ACTAMERICA_Merge/data/nc/ACTAMERICA-mrgPFP-b200_merge_20180508_R0.nc'

test_out_dir = '/home/jnd/Documents/missions/evs_mission_support/netcdf_services/resources/netcdf_headers_parsed/'

# Get output paths.
output_file_json = construct_PATH_OUTPUT(test_in_nc, test_out_dir, "json")
output_file_yaml = construct_PATH_OUTPUT(test_in_nc, test_out_dir, "yaml")

# Write the structure file.
get_netcdf_file_structure(
    input_path_netcdf=test_in_nc, 
    PATH_OUTPUT_json=output_file_json, 
    PATH_OUTPUT_yaml=output_file_yaml
)

In [44]:
!ls 


actamerica_config.json	icartt_headers_parsed  netcdf_headers_parsed


In [ ]:
multi1 = [i for i in icartts if "_L1" in i]
multi2 = [i for i in icartts if "_L2" in i]
print(multi1)
print(multi2)